In [1]:
from datetime import datetime

import pandas as pd
import psycopg2
from psycopg2.extras import Json

/home/antonio/.virtualenvs/myvirtualenv/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
df3600 = pd.read_csv('data/3600_tasks.csv', names=['sentence'], header=-0)

In [3]:
SIZE = 100
for i in range(0, len(df3600), SIZE):
    df3600[i:i+SIZE].to_csv('data/parts/{}_{}.csv'.format(i, i + SIZE - 1), index=False)

In [4]:
def columns2fields(columns):
    columns_str = '('
    for c in columns:
        columns_str += c + ', '
    columns_str = columns_str[:-2] + ')'
    return columns_str

def vformats():
    return '(%s, %s, %s, %s, %s, %s, %s, %s, %s)'

columns = [
#     'id',
    'created',
    'project_id',
    'state',
    'quorum',
    'calibration',
    'priority_0',
    'info',
    'n_answers',
    'fav_user_ids',
]

fields = columns2fields(columns)
formats = vformats()

In [5]:
conn = psycopg2.connect("dbname='pybossa' user='pybossa' host='localhost' port='5432' password='tester'")
cur = conn.cursor()

In [6]:
cur.execute('DELETE FROM task;')
cur.execute('DELETE FROM task_run;')
cur.execute('ALTER SEQUENCE task_id_seq RESTART WITH 1;')
cur.execute('ALTER SEQUENCE task_run_id_seq RESTART WITH 1;')

In [7]:
sql = "INSERT INTO task " + fields + " VALUES " + formats

for i in range(3600):
    values = (
#         i + 1,
        datetime.today(),
        1,
        'ongoing',
        0,
        0,
        0,
        Json({'sentence': df3600.loc[i].sentence}),
        3,
        None,
    )
    cur.execute(sql, values)

In [8]:
conn.commit()